In [1]:
#Import all the libraries
import pandas as pd
import copy
import random as rn
import time as tm
import numpy as np
import math
from inspect import currentframe, getframeinfo  #temp_dict,getframeinfo(currentframe()).lineno

    # Read the Metadata File
Table_Metadata_df = pd.read_excel('C:\Work\Mock Data Generator\Mock Data Generator - Metadata File.xlsx', sheet_name='Table Metadata',keep_default_na=False)
Column_Metadata_df = pd.read_excel('C:\Work\Mock Data Generator\Mock Data Generator - Metadata File.xlsx', sheet_name='Column Metadata',keep_default_na=False)

# The Below piece of code is used to handle nulls in individual columns which can lead to error
# Code starts here


# Code Ends Here
All_Table_Data_Dict = {}
All_Table_Key_Dict  = {}
Distinct_Value_Set = {}


#Split table into Dim/Fact Dictionary
Dim_Tables = Column_Metadata_df[Column_Metadata_df['Dim or Fact'] == '1 Dim']
Fact_Tables = Column_Metadata_df[Column_Metadata_df['Dim or Fact'] == '2 Fact']



##########################################################################################################################

#This function will help us to create a truly random number generator
def sys_rand_seed():
    return int(tm.time()*100000000000)%100000000000

def Create_ID_Column(srow): 
    
    id_df =  {} 
    if(srow["Key type PK or FK"] == 'PK'):
        for index in range(srow["No of Rows"]):
            id_df[index] = index + 1 
    else:
        print('Design for ID != PK is pending',getframeinfo(currentframe()).lineno)
    
    return id_df

#-------------------------------------------------------------------------------------------------------------------------#

def Generate_Name(dim_df,srow):
    for index in range(srow["No of Rows"]):
        dim_df[index] = 'FN'+str(index + 1) + ' ' + 'LN' + str(index + 1)

def Generate_Age(dim_df,srow):
    
    min_age = srow["Min Value"] if srow["Min Value"] !='' or type(srow["Min Value"]) !=str else 18
    max_age = srow["Max Value"] if srow["Max Value"] !='' or type(srow["Max Value"]) !=str else 70
    
    for index in range(srow["No of Rows"]):
        rn.seed(sys_rand_seed()+index)
        dim_df[index] = int(rn.randint((min_age),(max_age)))


#-------------------------------------------------------------------------------------------------------------------------#

def Create_Dim_Column(srow, nRows = None): 
    """
    The Create_Dim_Columnn() needs to handle the below conditions
    1. Prefix & Suffix - Done
    2. Leading 0's - Done
    3. Using FK to fetch data from PK when needed - Done
    4. If no suffix or Prefix value is present, use the column name - Done
    5. There is a hierarchy or relation between 2 dimension - for After Fact Table is handled
    """
    
    #--------------------------------------------------------------------------------------------------#
    #Space for variable declaration
    _is_lookup = 0
    prefix = srow['S or P Value'] if srow['Suffix or Prefix'] == 'P' else ''
    suffix = srow['S or P Value'] if srow['Suffix or Prefix'] == 'S' else ''
    default_label = srow["Column Name"]
    lenght = srow["Length of id with preceding zero"] if type(srow["Length of id with preceding zero"]) != str else 0
    min_value = srow["Min Value"] if type(srow["Min Value"]) != str else 0
    max_value = srow["Max Value"] if type(srow["Max Value"]) != str else srow["No of Rows"]
    total_rows = nRows if nRows != None else srow["No of Rows"]
    dim_df = {} 
    

    
    #--------------------------------------------------------------------------------------------------#
    
    
    #Handle leading zero or scenario where length is defined
    if(srow["Suffix or Prefix"].upper() == 'P'   or   srow["Suffix or Prefix"].upper() == 'S'   or   lenght > 0):
        if((len(srow["S or P Value"])+len(str(max_value))) > lenght): #Scenario where s/p value is larger than the total lenght of the value
            for index in range(total_rows):
                
                dim_df[index] = prefix + str(index + 1) + suffix
                
        elif(len(srow["S or P Value"]) == 0 or len(srow["Suffix or Prefix"]) == 0 ): #Scenario if no suffix or prefix is present
            for index in range(total_rows):
                dim_df[index] = default_label + str(min_value + index + 1)
                
        elif(len(srow["S or P Value"])+len(str(max_value)) <= lenght): #Preceding zero cases with suffix and prefix 
            for index in range(total_rows):
                rem_zero = lenght - (len(srow["S or P Value"])+ len(str(min_value + index + 1)))
                zero_str = str(pow(10,rem_zero))[(rem_zero*-1):]
                dim_df[index] = prefix + zero_str + str(min_value + index + 1) + suffix
    
    #--------------------------------------------------------------------------------------------------#
    
    elif(srow["Functional Category"] != ''):
        cat = srow["Functional Category"]
        if(cat == 'Name'):
            Generate_Name(dim_df,srow)
        elif(cat == 'Age'):
            Generate_Age(dim_df,srow)
    
    #Scenario where PK or FK is present        
    elif(srow["Key type PK or FK"] != ''): 
        if(srow["Key type PK or FK"] == 'FK'):
            
            parent_table = str(srow["Parent Column ID"]).split(".")[0]
            parent_column = str(srow["Parent Column ID"]).split(".")[1]
            
            #This function is used to get the min and max value in a column
            
            if parent_table in All_Table_Key_Dict:
                # It will get the values from the reference table which will mostly be Dim Tables
                min_index = 1
                max_index = max(All_Table_Key_Dict[parent_table][parent_column].values())
                
            else:
                Create_Dim_Table(parent_table)
                min_index = 1
                max_index = All_Table_Key_Dict[parent_table][parent_column].max()
            for index in range(total_rows): 
                # This loop will fill the data frame with the total number of rows as defined for the table
                rn.seed(sys_rand_seed()+index)
                dim_df[index] = rn.randint(min_index,max_index)
                
        elif(srow["Key type PK or FK"] == 'PK'):
            dim_df = Create_ID_Column(srow)
    else:
        for index in range(total_rows):
            dim_df[index] = default_label + str(min_value + index + 1)
    
    return dim_df
    
    
    #---------------------------------------------------------------------------------------------------#
    
    
    
#-------------------------------------------------------------------------------------------------------------------------#
    
def Create_Fact_Column(srow): #This will create a Fact Column also handle the Model Selection an applying that logic to the data
    fact_df = {} 
    
    min_value = srow["Min Value"]
    max_value = srow["Max Value"]
    
    for index in range(srow["No of Rows"]):
        rn.seed(sys_rand_seed()+index)
        fact_df[index] = int(rn.randint(min_value,max_value))

    return fact_df
    
   
#----------------------------------------------------------------------------------------------------------------------#
########################################################################################################################
def Create_Hier_Columns(srow):
    counter = 1      #Need to add comment for the counter
    max_value = 0 
    temp_dict = {} 
    total_row = srow['No of Rows'].unique()[0]
    temp_df = {} 
    srow = srow.sort_values('Hierarchy Rank')
    #Create a dictionary with the column names
    #Use this dictionary to fill in and create the actual database
    for index, row in srow.iterrows():
        temp_dict[str(row["Column Name"])] = Create_Dim_Column(row,nRows = row['Number of Unique Values'])
    
    
    for key in temp_dict:
        
        temp_df[key]={}
        min_value = 1
        max_value = (len(temp_dict[key]))
        
        if (    max_value < total_row     or     max_value > total_row    ): # This loop is for parent columns mostly
            
            for index in range(total_row):
                rn.seed(  sys_rand_seed()   +   index  )
                lkup_index = int( rn.randint(min_value , max_value) - 1 )
                temp_df[key][index] = temp_dict[key][lkup_index]
        
        else:
            for index in range(total_row):
                temp_df[key][index] = temp_dict[key][index]
    return temp_df



########################################################################################################################
#-------------------------------------------------------------------------------------------------------------------------#
#This function will be executed first and will create dimension table
def Create_Dim_Table(table_name): #Converting DF to Dict pending
    temp_meta = Dim_Tables[Dim_Tables['Table Name'] == table_name]
    temp_df = {} 
    
    for index, row in temp_meta.iterrows():
        
        if(row['Structural Category'] == 'ID'):
            temp_df[row['Column Name']] = Create_ID_Column(row) 
            
        elif(row['Structural Category'] == 'Dimension'):
            temp_df[row['Column Name']] = Create_Dim_Column(row)
        
        elif(row['Structural Category'] == 'Hierarchy'):
            
            if row['Column Name'] in temp_df.keys():
                continue
            
            else:
                hier_temp = Create_Hier_Columns(Dim_Tables[Dim_Tables['Hierarchy Name'] == row['Hierarchy Name']])
                for key in hier_temp:
                    temp_df[key] = hier_temp[key]
        
        else:
            print("End of loop or exception")
    return temp_df
    

#-------------------------------------------------------------------------------------------------------------------------#    

def Generate_Unique_Set()



#-------------------------------------------------------------------------------------------------------------------------#

#This function will be executed first and will create Fact table using the data from dim Table
def Create_Fact_Table(table_name): #Converting DF to Dict pending
    temp_meta = Fact_Tables[Fact_Tables['Table Name'] == table_name]
    temp_df =  {} 
    
    
    for index, row in temp_meta.iterrows():
        
        if(row['Structural Category'] == 'Fact'):
            temp_df[row['Column Name']] = Create_Fact_Column(row)
        
        elif(row['Structural Category'] == 'Dimension'):
            temp_df[row['Column Name']] = Create_Dim_Column(row)
        
        else:
            print("End of loop or exception",)
    return temp_df


#-------------------------------------------------------------------------------------------------------------------------#

for a in Dim_Tables["Table Name"].unique():
    All_Table_Key_Dict[a] = Create_Dim_Table(a)
    Generate_Unique_Set(All_Table_Key_Dict[a])


for a in Fact_Tables["Table Name"].unique():
    All_Table_Key_Dict[a] = Create_Fact_Table(a)

#for key in All_Table_Key_Dict:
#    print(key, ':\n',All_Table_Key_Dict[key],end = '\n\n\n', sep = '')

In [2]:
#Create Distinct value set

temp_df1 = {}
#This logic will be useful for replacing the values to a new value
for col in All_Table_Key_Dict['Product']:
    temp_df1[col] = {}
    for index, value in All_Table_Key_Dict['Product'][col].items():
        #print(index , value)
        temp_df1[col][value] = 1

        
#this code will make sure that the values are unique
for col in All_Table_Key_Dict['Product']:
    temp_df1[col] = {}
    counter = 0
    for index, value in All_Table_Key_Dict['Product'][col].items():
        if value in temp_df1[col].values():
            continue
        else:
            temp_df1[col][counter] = value
            counter += 1

print(temp_df1)

{'PID': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10}, 'PCat': {0: 'CAT3', 1: 'CAT1', 2: 'CAT2'}, 'PName': {0: 'Prod001235', 1: 'Prod001236', 2: 'Prod001237', 3: 'Prod001238', 4: 'Prod001239', 5: 'Prod001240', 6: 'Prod001241', 7: 'Prod001242', 8: 'Prod001243', 9: 'Prod001244'}}


temp = {}
for key,value in All_Table_Key_Dict['Product']['PName'].items():
    print('Current value at index {} is :{}. Change to:'.format(key,value),end ='')
    temp[key] = input('')

print('\nAll values have been modified!!')
    